In [1]:
# base libraries
import copy
import gc
import multiprocessing
from multiprocessing import Pool
import os
from pathlib import Path
import sys
import time

# installed libraries
import cv2
import imageio
import numpy as np
import numpy.matlib
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pandas as pd
import scipy
import scipy.signal
import sklearn as sk
import sklearn.decomposition
import skimage.draw

# my libraries
sys.path.append('/Users/akshay/Documents/Harvard/')
from basic_neural_processing_modules import *

In [2]:
## == IMPORT DATA ==
dir_s2p = Path('/Users/akshay/Documents/Harvard/suite2p/plane0')

F = np.load(dir_s2p / 'F.npy') # masks multiplied by data
Fneu = np.load(dir_s2p / 'Fneu.npy') # estimated neuropil signal (Fns = F - Fneu; Fo = ptile(Fns, 30); dFoF=Fns-Fo/Fo)
iscell = np.load(dir_s2p/'iscell.npy') # probability and bool of each roi
ops = np.load(dir_s2p / 'ops.npy', allow_pickle=True) # parameters for the suite2p
spks = np.load(dir_s2p / 'spks.npy') # blind deconvolution
stat = np.load(dir_s2p / 'stat.npy', allow_pickle=True) # statistics for individual neurons 

num_frames_S2p = F.shape[1]